In [7]:
import numpy as np
import pandas as pd
import pickle

from fbprophet import Prophet

import arima_gru
import validation

In [6]:
date_ranges = [
	pd.date_range(start='2014-03-17', end='2014-06-17').tolist(),
	pd.date_range(start='2014-06-18', end='2014-09-17').tolist(),
	pd.date_range(start='2014-09-18', end='2014-12-17').tolist(),
	pd.date_range(start='2014-12-18', end='2015-03-17').tolist(),

	pd.date_range(start='2015-03-18', end='2015-06-17').tolist(),
	pd.date_range(start='2015-06-18', end='2015-09-17').tolist(),
	pd.date_range(start='2015-09-18', end='2015-12-17').tolist(),
	pd.date_range(start='2015-12-18', end='2016-03-17').tolist(),

	pd.date_range(start='2016-03-18', end='2016-06-17').tolist(),
	pd.date_range(start='2016-06-18', end='2016-09-17').tolist(),
	pd.date_range(start='2016-09-18', end='2016-12-17').tolist(),
	pd.date_range(start='2016-12-18', end='2017-03-17').tolist(),

	pd.date_range(start='2017-03-18', end='2017-06-17').tolist(),
	pd.date_range(start='2017-06-18', end='2017-09-17').tolist(),
	pd.date_range(start='2017-09-18', end='2017-12-17').tolist(),
	pd.date_range(start='2017-12-18', end='2018-03-17').tolist(),

	pd.date_range(start='2018-03-18', end='2018-06-17').tolist(),
	pd.date_range(start='2018-06-18', end='2018-09-17').tolist(),
	pd.date_range(start='2018-09-18', end='2018-12-17').tolist(),
	pd.date_range(start='2018-12-18', end='2019-03-15').tolist()
]

In [3]:
with open('gru_optimized.pickle', 'rb') as p:
    gru_model = pickle.load(p)

In [4]:
gru_model

In [5]:
predictions = validation.make_predictions('SPY', gru_model)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


AttributeError: 'DatetimeIndex' object has no attribute 'index'

In [35]:
predictions_csv = pd.read_csv('/home/ubuntu/csvs/predictions.csv', parse_dates=True, infer_datetime_format=True)
predictions_csv = predictions_csv.set_index('date')
predictions_csv.index = pd.to_datetime(predictions_csv.index)

etfs = pd.read_csv('/home/ubuntu/csvs/etfs_pivot.csv', parse_dates=True, infer_datetime_format=True)
etfs = etfs.set_index('date')
etfs.index = pd.to_datetime(etfs.index)

macro = pd.read_csv('/home/ubuntu/csvs/macro_pivot.csv', parse_dates=True, infer_datetime_format=True)
macro = macro.set_index('date')
macro.index = pd.to_datetime(macro.index)

In [15]:
ts_date_range = pd.date_range(start=date_ranges[0][0], end=date_ranges[15][-1])
etf_df = pd.DataFrame(etfs['SPY'])
true_price = etf_df[etf_df.index.isin(ts_date_range)]
true_price = true_price.reset_index()
true_price.columns = ['ds', 'y']

In [16]:
arima_model = Prophet()
arima_model.fit(true_price)

future_date_range = [len(date_range) for date_range in date_ranges[16:]]

future = arima_model.make_future_dataframe(periods = sum(future_date_range))
forecast = arima_model.predict(future)

prediction_date_range = pd.date_range(start=date_ranges[16][0], end=date_ranges[-1][-1])
predicted_prices = np.array(forecast[forecast['ds'].isin(prediction_date_range)]['yhat'])
predictor_year = pd.date_range(start = date_ranges[12][0], end = date_ranges[15][-1])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [26]:
macro_predict = macro[macro.index.isin(predictor_year)]
macro_array = macro_predict.values
macro_predict = np.reshape(macro_array, (macro_array.shape[0], macro_array.shape[1], 1))
predicted_residuals = gru_model.predict(macro_predict).tolist()

In [33]:
predicted_residuals.pop()
predicted_residuals.pop()

final_predictions = (np.array(predicted_prices) + np.array(predicted_residuals)).tolist()

predictions_csv['SPY'] = final_predictions

predictions_csv.to_csv('/home/ubuntu/csvs/predictions.csv')

ValueError: Length of values does not match length of index

In [37]:
predicted_prices

array([272.16575597, 272.18516456, 272.16004489, 272.28383205,
       272.19513395, 272.21194491, 272.10869016, 272.02124129,
       272.01112533, 271.96614223, 272.07991279, 271.99111007,
       272.01766595, 271.93380976, 271.87507979, 271.90253027,
       271.90334928, 272.07040961, 272.04150591, 272.13357001,
       272.11972135, 272.13429323, 272.23705759, 272.31386336,
       272.55621024, 272.60051091, 272.76233295, 272.81347706,
       272.88703142, 273.04162461, 273.16209106, 273.43907046,
       273.50829323, 273.68484277, 273.74025032, 273.80756184,
       273.94559857, 274.03962511, 274.28094592, 274.30618244,
       274.43152113, 274.42978885, 274.43549487, 274.50906177,
       274.53745959, 274.71376483, 274.67639717, 274.74332539,
       274.68909955, 274.64985003, 274.68747662, 274.69024318,
       274.85230038, 274.81289055, 274.89052696, 274.86000534,
       274.85738909, 274.94419166, 275.00797219, 275.24186699,
       275.2838109 , 275.45074055, 275.51563717, 275.61

In [39]:
predicted_residuals = (predicted_residuals.reshape(predicted_residuals.shape[1], predicted_residuals.shape[0])[0]).tolist()

AttributeError: 'list' object has no attribute 'reshape'

In [40]:
predicted_residuals

[[1.6405079364776611],
 [1.6398569345474243],
 [1.6392173767089844],
 [1.6385565996170044],
 [1.6379032135009766],
 [1.6371784210205078],
 [1.6364505290985107],
 [1.6357150077819824],
 [1.6349753141403198],
 [1.6342259645462036],
 [1.633466362953186],
 [1.6326884031295776],
 [1.6319777965545654],
 [1.6312731504440308],
 [1.6305536031723022],
 [1.6303688287734985],
 [1.630168080329895],
 [1.6299586296081543],
 [1.6297332048416138],
 [1.6295106410980225],
 [1.6292805671691895],
 [1.629042387008667],
 [1.6287556886672974],
 [1.6284509897232056],
 [1.6281369924545288],
 [1.6278026103973389],
 [1.6274735927581787],
 [1.627144694328308],
 [1.6267940998077393],
 [1.626419186592102],
 [1.6260333061218262],
 [1.62562894821167],
 [1.6252082586288452],
 [1.624812364578247],
 [1.6243985891342163],
 [1.623966097831726],
 [1.6235629320144653],
 [1.6231459379196167],
 [1.6227190494537354],
 [1.6222751140594482],
 [1.621840000152588],
 [1.6214098930358887],
 [1.6209636926651],
 [1.6205029487609863],
 

In [41]:
resids = []

for resid in predicted_residuals:
    resids.append(resid[0])

In [42]:
resids

[1.6405079364776611,
 1.6398569345474243,
 1.6392173767089844,
 1.6385565996170044,
 1.6379032135009766,
 1.6371784210205078,
 1.6364505290985107,
 1.6357150077819824,
 1.6349753141403198,
 1.6342259645462036,
 1.633466362953186,
 1.6326884031295776,
 1.6319777965545654,
 1.6312731504440308,
 1.6305536031723022,
 1.6303688287734985,
 1.630168080329895,
 1.6299586296081543,
 1.6297332048416138,
 1.6295106410980225,
 1.6292805671691895,
 1.629042387008667,
 1.6287556886672974,
 1.6284509897232056,
 1.6281369924545288,
 1.6278026103973389,
 1.6274735927581787,
 1.627144694328308,
 1.6267940998077393,
 1.626419186592102,
 1.6260333061218262,
 1.62562894821167,
 1.6252082586288452,
 1.624812364578247,
 1.6243985891342163,
 1.623966097831726,
 1.6235629320144653,
 1.6231459379196167,
 1.6227190494537354,
 1.6222751140594482,
 1.621840000152588,
 1.6214098930358887,
 1.6209636926651,
 1.6205029487609863,
 1.620033621788025,
 1.620091199874878,
 1.6201372146606445,
 1.6202278137207031,
 1.6202